In [1]:
from preprocessing.pipeline import get_preprocessed_data, query_data, preprocess_311, process_crime_data
import pandas as pd
import pathlib
import copy


In [2]:
year = 2017
preprocessed_dir = pathlib.Path(f"../data/combined/{year}")

# try:
#     df = pd.read_parquet(preprocessed_dir)
#     assert len(df) != 0
# except AssertionError:
#     df = get_preprocessed_data(
#         start_date=year,
#         end_date=year+1,
#         opened_created_add=['borough', 'precinct', 'complaint_type'],
#     )


In [3]:
unproc_dir = pathlib.Path(rf"../data/temp/{year}")
try:
    df_311 = pd.read_parquet(unproc_dir/"311")
    df_crime = pd.read_parquet(unproc_dir/"crime")
    assert len(df_311) != 0
except AssertionError:
    df_311, df_crime = query_data(
        start_date=year,
        end_date=year+1,
    )
    df_311:pd.DataFrame
    df_311.to_parquet(unproc_dir/"311")
    df_crime.to_parquet(unproc_dir/"311")

In [4]:
df_crime_proc = process_crime_data(
    copy.deepcopy(df_crime),
    start_date=2017,
    sector_id=True
)
df_crime_proc

idx                        string[pyarrow]
report_date         timestamp[ns][pyarrow]
precinct                   string[pyarrow]
latitude                   double[pyarrow]
longitude                  double[pyarrow]
general_code               string[pyarrow]
specific_code              string[pyarrow]
attempted                  string[pyarrow]
crime_degree               string[pyarrow]
borough                    string[pyarrow]
relative_loc               string[pyarrow]
loc_description            string[pyarrow]
park_name                  string[pyarrow]
housing_code               string[pyarrow]
suspect_age                string[pyarrow]
suspect_race               string[pyarrow]
suspect_sex                string[pyarrow]
victim_age                 string[pyarrow]
victim_race                string[pyarrow]
victim_sex                 string[pyarrow]
crime_close_date    timestamp[ns][pyarrow]
crime_date          timestamp[ns][pyarrow]
dtype: object
Adding sector column to Crime Data...
Vi

c:\Users\Morri\.conda\envs\capstone\lib\site-packages\distributed\client.py:3163: UserWarning: Sending large graph of size 12.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


,idx,report_date,precinct,latitude,longitude,general_code,specific_code,attempted,crime_degree,borough,...,park_name,housing_code,suspect_age,suspect_race,suspect_sex,victim_age,victim_race,victim_sex,crime_close_date,crime_date
0,164730421,2017-05-11 00:00:00,47,40.877995,-73.846889,114,263,COMPLETED,FELONY,BRONX,...,(null),(null),(null),(null),(null),UNKNOWN,UNKNOWN,E,<NA>,2017-05-01 10:41:00
1,171684331,2017-11-18 00:00:00,47,40.868320,-73.833159,114,264,COMPLETED,FELONY,BRONX,...,(null),(null),(null),(null),(null),25-44,UNKNOWN,M,<NA>,2017-11-18 01:38:00
2,162582488,2017-03-07 00:00:00,10,40.746813,-74.007958,105,366,COMPLETED,FELONY,MANHATTAN,...,(null),(null),25-44,BLACK,M,25-44,WHITE,M,<NA>,2017-03-07 18:58:00
3,162390172,2017-02-26 00:00:00,72,40.662189,-73.997858,114,263,COMPLETED,FELONY,BROOKLYN,...,(null),(null),UNKNOWN,UNKNOWN,U,UNKNOWN,UNKNOWN,E,<NA>,2016-12-18 04:10:00
4,160222672,2017-01-12 00:00:00,120,40.635216,-74.106642,109,438,COMPLETED,FELONY,STATEN ISLAND,...,(null),(null),(null),(null),(null),45-64,WHITE,M,2017-01-11 23:30:00,2017-01-11 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469012,163224185,2017-03-28 00:00:00,113,40.687028,-73.755265,578,638,COMPLETED,VIOLATION,QUEENS,...,(null),(null),UNKNOWN,BLACK,M,25-44,BLACK,F,<NA>,2017-02-20 08:30:00
469013,168486539,2017-08-25 00:00:00,105,40.670177,-73.749448,578,638,COMPLETED,VIOLATION,QUEENS,...,(null),(null),25-44,BLACK,M,25-44,WHITE HISPANIC,F,<NA>,2017-08-25 06:15:00
469014,162109080,2017-02-21 00:00:00,103,40.709264,-73.775467,126,198,COMPLETED,FELONY,QUEENS,...,(null),(null),45-64,UNKNOWN,M,<18,UNKNOWN,F,2017-02-21 00:10:00,2017-02-21 00:00:00
469015,171879020,2017-11-22 00:00:00,113,40.679139,-73.760489,578,638,COMPLETED,VIOLATION,QUEENS,...,(null),(null),(null),(null),(null),25-44,AMERICAN INDIAN/ALASKAN NATIVE,M,2017-11-22 11:50:00,2017-11-22 10:00:00


In [5]:
df_crime_proc['sector']

KeyError: 'sector'

In [4]:
# df['closed_date'] = df['closed_date'].astype('timestamp[ns][pyarrow]')



df_311_proc = preprocess_311(
    copy.deepcopy(df_311),
    opened_created_add=['borough', 'precinct', 'complaint_type'],
    sectors=False,
    precincts=True
    )
df_311_proc


Checking dates...
Adding precincts...
View progress here http://127.0.0.1:8787/status


c:\Users\Morri\.conda\envs\capstone\lib\site-packages\distributed\client.py:3163: UserWarning: Sending large graph of size 35.61 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-04-08 10:01:46,008 - distributed.nanny - WARNING - Worker process still alive after 3.1999969482421875 seconds, killing
2024-04-08 10:01:46,055 - distributed.nanny - WARNING - Worker process still alive after 3.1999978637695317 seconds, killing
2024-04-08 10:01:46,057 - distributed.nanny - WARNING - Worker process still alive after 3.1999981689453127 seconds, killing
2024-04-08 10:01:46,058 - distributed.nanny - WARNING - Worker process still alive after 3.199998474121094 seconds, killing
2024-04-08 10:01:46,060 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing
2024-04-08 10:01:46,063 - distributed.nanny - WARNING - Worker process still alive after 3.1999987792968754 seconds, killing
2024-04-08 1

KeyboardInterrupt: 

In [13]:
df = _
temp = df.hours_to_complete
temp.dtype

duration[ns][pyarrow]

In [24]:
import numpy as np
# temp.astype('timedelta64[ns]') / np.timedelta64(1, 'h')

pd.to_timedelta(temp)

TypeError: Invalid type for timedelta scalar: <class 'pandas._libs.missing.NAType'>

In [16]:
import numpy as np
temp / np.timedelta64(1, 'h')

121        inf
122        inf
126        inf
127        inf
129        inf
          ... 
5128125    inf
5128126    inf
5128129    inf
5128130    inf
5128131    inf
Name: hours_to_complete, Length: 1495172, dtype: double[pyarrow]

In [4]:
from preprocessing.volume_feature import add_powerset_created
add_powerset_created(df,['borough', 'precinct', 'complaint_type'])

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Timestamp'

In [ ]:
try:
    preprocessed_dir.mkdir()
except:
    pass

In [ ]:
df.to_parquet(preprocessed_dir/f"{year}_311.pkt")
